# Chapter 2-Multi-arm Bandits

区分强化学习和其他学习的最重要的特征是其评估要采取的动作而非直接给出正确的动作。
    - evaluative feedback：indicate how good the action taken is. 完全依赖于采取的动作
    - instructive feedback：indicate the correct action to take. 和采取的动作相互独立

### 2.1 A k-Armed Bandit Problem
- k-armed bandit problem
    - 问题描述：你需要重复从k个不同的option（或action）中做出选择。每次选择后你会收到一个从静态概率分布中抽取的numerical reward，该分布由你选择的action决定。你的目标是最大化一定时期内（譬如1000次动作选择）的期望总回报（expected total reward）
    - value of the action: $q_*(a)=E[R_t|A_t=a]$
        - 时刻$t$选取的action：$A_t$
        - $A_t$对应的reward：$R_t$
        - 假如我们确切的知道每个action的value，那么我们每次都会选取具有最大value的action，这个问题就是平凡的。因此我们假设我们并不能确定动作价值，而只能估计它，用$Q_t(a)\approx q_*(a)$表示动作$a$在时刻$t$的价值估计。
    - exploration & exploitation
        - greedy action：在任何时刻都至少有一个action的估计价值是 最大的，我们称其为greedy action。选择greedy action意味着你在利用动作价值的已知知识。
        - 当选择非贪婪动作时，你在探索，这使你可以改进非贪婪动作的价值估计。
        - Exploitation is the right thing to do to maximize the expected reward on the one step, but exploration may produce the greater total reward in the long run.
        - 本章中将展示一些简单的平衡（exploration和exploitation的）方法，它们比只进行exploit要好得多。

### 2.2 Action-Value Methods
- Sample-average method：以实际收到的reward均值作为action value
$$Q_t(a)=\frac{\sum_{i=1}^{t-1}{R_i\cdot 1_{A_i=a}}}{\sum_{i=1}^{t-1}{1_{A_i=a}}}$$
    - 随机变量$1_predicate$：当$predicate$为True时，变量为1，否则变量为0
    - 当上式分子为0时，$Q_t(a)$被设定为某个默认值。
    - 由大数定律可知，当分子趋近于无穷大时，$Q_t(a)$将收敛到$q_*(a)$

- Action selection rule
    - greedy action selection rule: 选择具有最大动作价值的动作。
    $$A_t = argmax_a Q_t(a)$$
    - $\epsilon$-greedy methods:在大部分时间选择greedy action，但是有$\epsilon$的概率从non-greedy actions中等概率的选择一个action。
        - 这种方法的一个优点是，当时间步趋近于无穷大时，每个action被采样的次数也将趋近于无穷大，从而保证所有的$Q_t(a)$收敛到$q_*(t)$。这当然也暗示了选到最优action的概率将受到到大于$1-\epsilon$     
        

### 2.3 Incremental Implementation
- Incremental fomulas for updating averages
$$Q_{n+1} = \frac{1}{n}\sum\limits_{i=1}^n{R_i}=Q_n + \frac{1}{n}\Big[R_n-Q_n\Big]$$
$$NewEstimate \Leftarrow OldEstimate+StepSize\Big[Target-OldEstimate\Big]$$
- **A simple bandit algorithm**
> Initialize, for a=1 to k:
    $$Q(a) \Leftarrow 0$$
    $$N(a) \Leftarrow 0$$
  Repeat forever:
    $$A \Leftarrow \begin{cases}
    argmax_a Q(a) & \quad \text{with probability } 1-\epsilon\\
    a\ random\ action  & \quad \text{with probability } \epsilon\\
  \end{cases}$$
  $$R \Leftarrow bandit(A)$$
  $$N(A) \Leftarrow N(A)+1$$
  $$Q(A) \Leftarrow Q(A)+\frac{1}{N(A)}\Big[R-Q(A)\Big]$$

### 2.4 Tracking a Nonstationary Problem
- 前面的averaging method适用于平稳环境（stationary environment）,对于非平稳情况来说，给近期的reward赋予更高的权重更合理一些。一种常见的做法是采用固定step size $\alpha$，可以证明这与指数平均等价(也被称为exponential recency-weighted average)。
$$Q_n = Q_n + \alpha\Big[R_n-Q_n\Big]=(1-\alpha)^nQ_1+\sum\limits_{i=1}^n{\alpha(1-\alpha)^{n-i}R_i}$$

- A well-known result in stochastic approximation theory gives us the conditions required to assure convergence with probability 1:
$$\sum\limits_{n=1}^\infty{a_n(a)}=\infty \quad\text{and}\quad \sum\limits_{n=1}^\infty{a_n^2(a)<\infty}$$
第一个条件保证步子足够大，最终可以克服任何初始条件或者随机波动，第二个条件保证最终时间步足够小从而可以实现收敛。
    - 若$\alpha_n(a)=\frac{1}{n}$，两个条件均成立
    - 若$\alpha_n(a)=\alpha$，即为固定值，则第二个调价是不满足的。这表明价值估计从未完全收敛，而是持续对最近接收的reward做出反应，这一点是非平稳环境想要的。
    - 同时满足两个条件的步长参数序列往往收敛非常慢，需要相当大的改动才能达到令人满意的收敛速率。其通常用在理论工作而非实际应用中和经验研究中。

### 2.5 Optimistic Initial Values

所有前面提到的方法都在一定程度上依赖于初始action-value estimates $Q_1(a)$。In the language of statistics, these methods are biased by their initial estimates.

对于sample-average method，一旦所有的动作都被至少选择一次，bias将消失。但是对于固定步长$\alpha$，bias是始终存在的，随着时间的演进而减小$\Big((1-\alpha)^nQ_1\Big)$。实际上，这种bias通常不是一个问题，甚至有时是有用的。
    - downside：initial estimate编程了一组需要由用户选取的参数。
    - upside：bias provide an easy way to supply some prior knowledge about what level of rewards can be expected
    
- **Initial action values可以被用作一种鼓励探索的简单方法。**
但初始action value远好于实际动过价值时（wildly optmistic），无论最开始选择哪儿action，reward都小于初始估计，从而将切换到其他action上，最终在价值估计收敛前，所有的动作都被试了很多次，即便总是选取greedy action也是如此。我们称这种鼓励探索的技术为<font color=#ff2222>**optimistic initial values**</font>。这种方法在stationary problems相当有效，但其并不是一个通用的技巧，譬如其不适合处理nonstationary problems，因为其鼓励探索的效果只在开始的时候有效，作用是暂时的。

### 2.6 Upper-Confidence-Bound Action Selection (UCB)

- Upper-Confidence-Bound Action Selection (UCB)<br>
    $\epsilon$-greedy action selection对所有的动作一视同仁。但似乎根据non-greedy action是实际最优动作的潜力来选择动作更合理，同时考虑价值估计离最大的距离以及估计的不确定度。
$$A_t = argmax_a\Big[Q_t(a)+c\sqrt{\frac{log t}{N_t(a)}}\Big]$$
    其中，$N_t(a)$表示时刻$t$之前动作$a$被选中的次数；$c>0$控制探索的力度。如果$N_t(a)=0$，那么动作$a$将被视为具有最大价值的动作，即优先选择从未被选取的动作进行探索。<br>
    <font color=#2222ff>所有的action最终都会被选到，但是随着时间的推移，那些具有更低价值估计或者已经被选很多次的动作被选到的频率将降低。</font>
    
- <font color=#ff2222>与$\epsilon$-greedy方法相比，UCB更难被推广到更一般的强化学习问题中。</font>其中一个困难是非平稳问题的处理，另一个困难是大状态空间的处理，尤其是本书Part II中讲到的的funciton approximation。

### 2.7 Gradient Bandit Algorithms
为每个动作$a$学习数值偏好(numerical preference)$H_t(a)$。preference越大，action被选到的概率越大。只有动作之间的相对偏好重
$$Pr\{A_t=a\} = \frac{e^{H_t(a)}}{\sum_{b=1}^ke^{H_t(b)}} = \pi_t(a)$$
一开始，所有动作的偏好相同（e.g.,$H_1(a)=0 \quad\forall a$），从而所有的动作被选到的概率相同。<br>
根据随机梯度上升的思想可以得到一个学习算法。在每一时间步，选择动作$A_t$，然后接收reward $R_t$，偏好通过下列公式更新：
$$H_{t+1}(A_t) = H_t(A_t) + \alpha(R_t-\bar{R}_t)\Big(1-\pi_t(A_t)\Big)$$
$$H_{t+1}(a) = H_t(a) - \alpha(R_t-\bar{R}_t)\pi_t(a)\quad\forall{a\neq A_t}$$
其中，$\alpha>0$是更新步长，$\bar{R}_t\in\mathbb{R}$是时刻$t$及之前的所有reward的均值，可以使用2.3节中的增量方法来进行计算。$\bar{R}_t$是reward比较的基准，当t时刻接收的reward $R_t$高于基准时，未来采取动作$A_t$的可能性将上升；而当reward低于基准时，非被选中动作被选中的可能性将上升，从而动作$A_t$未来被选中的可能性将下降。

### 2.8 Associative Search (Contextual Bandits)
